## Imports

In [1]:
import copy
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from config import config
import os
import sys
current_dir = os.getcwd()
path = "C:\\Users\\eirik\\Documents\\Master\\ISLBBNN\\islbbnn"
# path = "C:\\you\\path\\to\\islbbnn\\folder\\here"
os.chdir(path)
import plot_functions as pf
import pipeline_functions as pip_func
sys.path.append('networks')
from flow_net import BayesianNetwork
import torch.nn.functional as F

os.chdir(current_dir) # set the working directory back to this one 

CPUs are used!


# Problem description

Consits of 1095 datapoints, 77 proteins, and 7 different classes to predict. 

Classes:

* c-CS-s: control mice, stimulated to learn, injected with saline (9 mice)
* c-CS-m: control mice, stimulated to learn, injected with memantine (10 mice)
* c-SC-s: control mice, not stimulated to learn, injected with saline (9 mice)
* c-SC-m: control mice, not stimulated to learn, injected with memantine (10 mice)
* t-CS-s: trisomy mice, stimulated to learn, injected with saline (7 mice)
* t-CS-m: trisomy mice, stimulated to learn, injected with memantine (9 mice)
* t-SC-s: trisomy mice, not stimulated to learn, injected with saline (9 mice)
* t-SC-m: trisomy mice, not stimulated to learn, injected with memantine (9 mice)

Attained from this website: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0129126

# Batch size and parameters

In [2]:
# define parameters
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
post_train_epochs = config['post_train_epochs']
dim = config['hidden_dim']
num_transforms = config['num_transforms']
n_nets = config['n_nets']
lr = config['lr']
class_problem = config["class_problem"]
verbose = config['verbose']
save_res = config['save_res']
patience = config['patience']
a_prior = config['a_prior']
SAMPLES = 1



#---------DATA------------
df = pd.read_excel("Data_Cortex_Nuclear.xlsx")
X_pd = df.iloc[:,1:-4]
X_pd.fillna(X_pd.mean(),inplace=True)
X_original = X_pd.values
target = df.values[:, -1]
n, p = X_original.shape  # need this to get p 
y_original = np.zeros(len(target))
labeling = {}
for i, val in enumerate(np.unique(target)):
    labeling[i] = val
    y_original[target==val] = i

print(n,p)
print(labeling)

n_classes = len(labeling)
multiclass = n_classes > 1

BATCH_SIZE = int(n*0.8)
TEST_BATCH_SIZE = int(n*0.10) # Would normally call this the "validation" part (will be used during training)
VAL_BATCH_SIZE = int(n*0.10) # and this the "test" part (will be used after training)

TRAIN_SIZE = int(n*0.80)
TEST_SIZE = int(n*0.10) # Would normally call this the "validation" part (will be used during training)
VAL_SIZE = int(n*0.10) # and this the "test" part (will be used after training)

NUM_BATCHES = TRAIN_SIZE/BATCH_SIZE

assert (TRAIN_SIZE % BATCH_SIZE) == 0
assert (TEST_SIZE % TEST_BATCH_SIZE) == 0

1095 77
{0: 'c-CS-m', 1: 'c-CS-s', 2: 'c-SC-m', 3: 'c-SC-s', 4: 't-CS-m', 5: 't-CS-s', 6: 't-SC-m', 7: 't-SC-s'}


c:\Users\eirik\miniconda3\envs\master\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


# Sigmoid based network

## Seperate a test set for later

In [3]:
# Split keep some of the data for validation after training
X, X_test, y, y_test = train_test_split(
    copy.deepcopy(X_original), copy.deepcopy(y_original), test_size=0.10, random_state=42, stratify=y_original)

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

test_dat = torch.tensor(np.column_stack((X_test,y_test)),dtype = torch.float32)

## Train, validate, and test network

In [4]:
# select the device and initiate model

# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "mps")
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}

all_nets = {}
metrics_several_runs = []
metrics_median_several_runs = []
for ni in range(n_nets):
    post_train = False
    print('network', ni)
    # Initate network
    torch.manual_seed(ni+42)
    net = BayesianNetwork(dim, p, HIDDEN_LAYERS, classification=class_problem, num_transforms=num_transforms, a_prior=a_prior, n_classes=n_classes).to(DEVICE)
    alphas = pip_func.get_alphas_numpy(net)
    nr_weights = np.sum([np.prod(a.shape) for a in alphas])
    print(nr_weights)

    optimizer = optim.Adam(net.parameters(), lr=lr)
    
    all_nll = []
    all_loss = []

    # Split into training and test set
    X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=1/9, random_state=ni)#, stratify=y)
            
    train_dat = torch.tensor(np.column_stack((X_train,y_train)),dtype = torch.float32)
    val_dat = torch.tensor(np.column_stack((X_val,y_val)),dtype = torch.float32)
    
    # Train network
    counter = 0
    highest_acc = 0
    best_model = copy.deepcopy(net)
    for epoch in range(epochs + post_train_epochs):
        if verbose:
            print(epoch)
        nll, loss = pip_func.train(net, train_dat, optimizer, BATCH_SIZE, NUM_BATCHES, p, DEVICE, nr_weights, post_train=post_train, multiclass=multiclass)
        nll_val, loss_val, ensemble_val = pip_func.val(net, val_dat, DEVICE, verbose=verbose, reg=(not class_problem), post_train=post_train, multiclass=multiclass)
        if ensemble_val >= highest_acc:
            counter = 0
            highest_acc = ensemble_val
            best_model = copy.deepcopy(net)
        else:
            counter += 1
        
        all_nll.append(nll)
        all_loss.append(loss)

        if epoch == epochs-1:
            post_train = True   # Post-train --> use median model 
            for name, param in net.named_parameters():
                for i in range(HIDDEN_LAYERS+1):
                    #if f"linears{i}.lambdal" in name:
                    if f"linears.{i}.lambdal" in name:
                        param.requires_grad_(False)

        if counter >= patience:
            break
        
    all_nets[ni] = net 
    # Results
    metrics, metrics_median = pip_func.test_ensemble(all_nets[ni],test_dat,DEVICE,SAMPLES=10, reg=(not class_problem), multiclass=multiclass) # Test same data 10 times to get average 
    metrics_several_runs.append(metrics)
    metrics_median_several_runs.append(metrics_median)
    pf.run_path_graph(all_nets[ni], threshold=0.5, save_path=f"path_graphs/flow/prob/test{ni}_sigmoid", show=verbose)

if verbose:
    print(metrics)
m = np.array(metrics_several_runs)
m_median = np.array(metrics_median_several_runs)

network 0
11216
0
loss 164553.0
nll 1824.1126708984375
density 0.9837026928900907

val_loss: 162107.4219, val_nll: 265.0645, val_ensemble: 0.2727, used_weights_median: 11216

1
loss 164009.609375
nll 2170.796630859375
density 0.9829214980637125

val_loss: 162407.2812, val_nll: 280.3105, val_ensemble: 0.1636, used_weights_median: 11216

2
loss 164235.546875
nll 2130.0673828125
density 0.9821044557147292

val_loss: 160157.7812, val_nll: 225.7739, val_ensemble: 0.1727, used_weights_median: 11216

3
loss 161690.6875
nll 1759.8516845703125
density 0.9812496295902936

val_loss: 159371.1719, val_nll: 213.6569, val_ensemble: 0.3727, used_weights_median: 11216

4
loss 160857.59375
nll 1714.2587890625
density 0.9803550119077343

val_loss: 158749.0000, val_nll: 208.9959, val_ensemble: 0.2909, used_weights_median: 11216

5
loss 160181.28125
nll 1659.8544921875
density 0.9794184093266598

val_loss: 157947.0469, val_nll: 206.0579, val_ensemble: 0.2364, used_weights_median: 11216

6
loss 159357.79687

## Attain weigth graph

In [5]:
pf.run_path_graph_weight(net, save_path="path_graphs/flow/weight/temp_sigmoid", show=True, flow=True)

## Individual paths for each class

In [6]:
pf.plot_path_individual_classes(net, n_classes)

Used weights:  8
Used weights:  12
Used weights:  9
Used weights:  17
Used weights:  13
Used weights:  11
Used weights:  14
Used weights:  7


# ReLU based network

## Seperate a test set for later

In [7]:
# Split keep some of the data for validation after training
X, X_test, y, y_test = train_test_split(
    copy.deepcopy(X_original), copy.deepcopy(y_original), test_size=0.10, random_state=42, stratify=y_original)

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

test_dat = torch.tensor(np.column_stack((X_test,y_test)),dtype = torch.float32)

## Train, validate, and test network

In [8]:
# select the device and initiate model

# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "mps")
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}

all_nets = {}
metrics_several_runs = []
metrics_median_several_runs = []
for ni in range(n_nets):
    post_train = False
    print('network', ni)
    # Initate network
    torch.manual_seed(ni+42)
    #---------------------------
    # DIFFERENCE IS IN act_func=F.relu part
    net = BayesianNetwork(dim, p, HIDDEN_LAYERS, classification=class_problem, num_transforms=num_transforms, act_func=F.relu, a_prior=a_prior, n_classes=n_classes).to(DEVICE)
    #---------------------------
    alphas = pip_func.get_alphas_numpy(net)
    nr_weights = np.sum([np.prod(a.shape) for a in alphas])
    print(nr_weights)

    optimizer = optim.Adam(net.parameters(), lr=lr)
    
    all_nll = []
    all_loss = []

    # Split into training and test set
    X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=1/9, random_state=ni)#, stratify=y)
            
    train_dat = torch.tensor(np.column_stack((X_train,y_train)),dtype = torch.float32)
    val_dat = torch.tensor(np.column_stack((X_val,y_val)),dtype = torch.float32)
    
    # Train network
    counter = 0
    highest_acc = 0
    best_model = copy.deepcopy(net)
    for epoch in range(epochs + post_train_epochs):
        if verbose:
            print(epoch)
        nll, loss = pip_func.train(net, train_dat, optimizer, BATCH_SIZE, NUM_BATCHES, p, DEVICE, nr_weights, post_train=post_train, multiclass=multiclass)
        nll_val, loss_val, ensemble_val = pip_func.val(net, val_dat, DEVICE, verbose=verbose, reg=(not class_problem), post_train=post_train, multiclass=multiclass)
        if ensemble_val >= highest_acc:
            counter = 0
            highest_acc = ensemble_val
            best_model = copy.deepcopy(net)
        else:
            counter += 1
        
        all_nll.append(nll)
        all_loss.append(loss)

        if epoch == epochs-1:
            post_train = True   # Post-train --> use median model 
            for name, param in net.named_parameters():
                for i in range(HIDDEN_LAYERS+1):
                    #if f"linears{i}.lambdal" in name:
                    if f"linears.{i}.lambdal" in name:
                        param.requires_grad_(False)

        if counter >= patience:
            break
        
    all_nets[ni] = net 
    # Results
    metrics, metrics_median = pip_func.test_ensemble(all_nets[ni],test_dat,DEVICE,SAMPLES=10, reg=(not class_problem), multiclass=multiclass) # Test same data 10 times to get average 
    metrics_several_runs.append(metrics)
    metrics_median_several_runs.append(metrics_median)
    pf.run_path_graph(all_nets[ni], threshold=0.5, save_path=f"path_graphs/flow/prob/test{ni}_relu", show=verbose)

if verbose:
    print(metrics)
m = np.array(metrics_several_runs)
m_median = np.array(metrics_median_several_runs)

network 0
11216
0
loss 164552.8125
nll 1823.9256591796875
density 0.9837026928900907

val_loss: 162120.6250, val_nll: 234.7529, val_ensemble: 0.2455, used_weights_median: 11216

1
loss 163776.46875
nll 1895.4864501953125
density 0.9829215098666652

val_loss: 162456.9219, val_nll: 226.3270, val_ensemble: 0.2000, used_weights_median: 11216

2
loss 163992.53125
nll 1775.3282470703125
density 0.9821044582815126

val_loss: 160173.3750, val_nll: 227.3213, val_ensemble: 0.2182, used_weights_median: 11216

3
loss 161709.828125
nll 1762.025146484375
density 0.9812495935330973

val_loss: 159363.7344, val_nll: 206.8702, val_ensemble: 0.2273, used_weights_median: 11216

4
loss 160818.234375
nll 1671.7957763671875
density 0.9803549409093343

val_loss: 158685.7812, val_nll: 200.4535, val_ensemble: 0.2455, used_weights_median: 11216

5
loss 160072.125
nll 1600.3111572265625
density 0.9794183133937918

val_loss: 157884.7188, val_nll: 193.5798, val_ensemble: 0.2818, used_weights_median: 11216

6
loss 1

## Attain weight graph

In [9]:
pf.run_path_graph_weight(net, save_path="path_graphs/flow/weight/temp_relu", show=True, flow=True)

## Individual paths

In [10]:
pf.plot_path_individual_classes(net, n_classes)

Used weights:  9
Used weights:  11
Used weights:  7
Used weights:  12
Used weights:  9
Used weights:  13
Used weights:  12
Used weights:  6


In [11]:
net.eval()
net(train_dat[0:7, :-1], ensemble=False)

tensor([[-2.7387e+01, -2.1917e+01, -2.6065e+01, -8.9620e+00, -9.4833e+00,
         -1.2214e+01, -2.5696e+01, -2.0931e-04],
        [-2.5233e+00, -1.4265e-01, -2.2439e+01, -1.7068e+01, -2.9439e+00,
         -2.6922e+01, -3.1175e+01, -9.3124e+00],
        [-8.9109e+00, -3.0058e-01, -1.4194e+01, -2.8331e+01, -5.3219e+00,
         -1.3681e+00, -1.1939e+01, -1.8854e+01],
        [-1.0128e+00, -1.8685e+01, -4.2171e+00, -6.0816e-01, -3.5175e+00,
         -3.6128e+01, -6.9749e+00, -3.0554e+00],
        [-1.2970e+01, -1.7310e+01, -8.3935e+00, -6.7718e+00, -2.8353e+00,
         -1.1326e+01, -1.3487e+01, -6.1967e-02],
        [-1.0261e+01, -6.3533e+00, -3.0198e+01, -3.4110e+01, -6.6936e+00,
         -3.1829e-03, -3.5924e+01, -8.7219e+00],
        [-9.1388e-02, -4.3988e+00, -1.4474e+01, -9.8577e+00, -2.5924e+00,
         -1.0900e+01, -3.1387e+01, -8.9323e+00]],
       grad_fn=<LogSoftmaxBackward0>)

In [12]:
train_dat

tensor([[0.0535, 0.0971, 0.3572,  ..., 0.6101, 0.5041, 7.0000],
        [0.0993, 0.1077, 0.6474,  ..., 0.2829, 0.6028, 0.0000],
        [0.1303, 0.1573, 0.5744,  ..., 0.3253, 0.5740, 1.0000],
        ...,
        [0.0508, 0.0880, 0.4796,  ..., 0.3174, 0.1975, 2.0000],
        [0.1072, 0.1769, 0.5322,  ..., 0.3253, 0.2822, 6.0000],
        [0.0584, 0.1261, 0.5026,  ..., 0.3285, 0.3237, 6.0000]])